# import bibliotéque

In [28]:
import numpy as np
import pandas as pd 
import json
import os

# Ouverture des fichier

In [29]:
def read_data_csv(data1,data2):
    books=pd.read_csv(data1)
    ratings=pd.read_csv(data2)
    return(books,ratings)
books_raw,ratings_raw=read_data_csv("books_data_t.csv","Books_rating_t.csv")

In [30]:
Generation_folder = 'generated' 

# Netoyage des données
## Ratings

In [31]:
ratings_raw.head()

,Id,Titre,Prix,User_id,Nom lecteur,revue/utilité,revue/score,revue/heure,revue/résumé,revue/texte
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


### Valeurs manquantes

In [32]:
print(ratings_raw.isnull().sum())
print("---------------------------------------------")
#on ne peut pas travailler avec des valeurs manquantes dans les commentaires
ratings = ratings_raw.dropna(subset=['revue/résumé'])
ratings = ratings.dropna(subset=['revue/texte'])


Id                     0
Titre                208
Prix             2518829
User_id           561787
Nom lecteur       561886
revue/utilité          0
revue/score            0
revue/heure            0
revue/résumé          38
revue/texte            8
dtype: int64
---------------------------------------------


In [33]:
#on va supprimer les colonnes inutiles
ratings = ratings.drop(columns=['Prix'])  
#ratings.head()

## transformation en numérique de la colonne revue/utilité

In [34]:
def utilite_stringto_int(vale,teste=False):
    for i in range(len(vale)):
        if teste:
            print("indice",i,"valeur",vale[i])
            if i!=0:
                print("derniere valeur traiter")
                print(vale[i-1])
        if vale[i][0]=='0':
            vale[i]=0
        else:
            vale[i] = eval(vale[i])
    return(vale)

valtest=np.array(ratings["revue/utilité"])
valtest=utilite_stringto_int(valtest,teste=False)
ratings["utilite_num"]=valtest

## Book

In [35]:
books_raw.head()

,Titre,Description,Auteurs,Image,Lien Google,Editeur,Date publication,infoLink,Genre,Nb scores
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,01/01/2005,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,01/03/2003,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


In [36]:
#extraire seulement le genre principal
books_raw['Genre'] = books_raw['Genre'].str.strip('[]').str.replace("'","")
books_raw['Auteurs'] = books_raw['Auteurs'].str.strip('[]').str.replace("'","")

In [37]:
books_raw['year'] = books_raw['Date publication'].astype(str).str.extract(r'(\d{4})')

In [38]:
books = books_raw.drop(columns=['Description'])  
books = books.drop(columns=['Image'])
books = books.drop(columns=['Lien Google'])
books = books.drop(columns=['Date publication'])
books = books.drop(columns=['infoLink'])

In [39]:
books.head(11)

,Titre,Auteurs,Editeur,Genre,Nb scores,year
0,Its Only Art If Its Well Hung!,Julie Strain,NaN,Comics & Graphic Novels,NaN,1996
1,Dr. Seuss: American Icon,Philip Nel,A&C Black,Biography & Autobiography,NaN,2005
2,Wonderful Worship in Smaller Churches,David R. Ray,NaN,Religion,NaN,2000
3,Whispers of the Wicked Saints,Veronica Haddon,iUniverse,Fiction,NaN,2005
4,"Nation Dance: Religion, Identity and Cultural ...",Edward Long,NaN,NaN,NaN,2003
5,The Church of Christ: A Biblical Ecclesiology ...,Everett Ferguson,Wm. B. Eerdmans Publishing,Religion,5.0,1996
6,The Overbury affair (Avon),Miriam Allen De Ford,NaN,NaN,NaN,1960
7,A Walk in the Woods: a Play in Two Acts,Lee Blessing,NaN,NaN,3.0,1988
8,Saint Hyacinth of Poland,Mary Fabyan Windeatt,Tan Books & Pub,Biography & Autobiography,NaN,2009
9,Rising Sons and Daughters: Life Among Japan's ...,Steven Wardell,Plympton PressIntl,Social Science,NaN,1995


# Statistiques des auteurs/genre/éditeur
On ne fais pas de statistique sur les titres car cela rique de biaser l'algorithme

In [40]:
data = books[['Titre','Auteurs','Genre','Editeur']].values
titre_info = {}
for i in range(len(data)):
    dict = {}
    dict['Auteurs'] = data[i][1]
    dict['Genre'] = data[i][2]
    dict['Editeur'] = data[i][3]
    titre_info[data[i][0]] = dict
#titre_info

In [41]:
#on sauvegarde le dictionnaire dans un ficher json

with open(os.path.join(Generation_folder, 'titre_info.json'), 'w') as fp:
    json.dump(titre_info, fp)

In [42]:
ratings_books = ratings[['Titre','revue/score']].values

In [43]:
ratings_books[:,1]=ratings_books[:,1].astype(float)    

In [44]:
author_stats = {}
genre_stats = {}
editor_stats = {}
for i in range(len(ratings_books)):
    titre = ratings_books[i][0]
    note = ratings_books[i][1]
    if titre  in titre_info:         
        auteurs = titre_info[titre]['Auteurs']
        genre = titre_info[titre]['Genre']
        editeur = titre_info[titre]['Editeur']
        if auteurs in author_stats:
            author_stats[auteurs].append(note)
        else:
            author_stats[auteurs] = [note]
        if genre in genre_stats:
            genre_stats[genre].append(note)
        else:
            genre_stats[genre] = [note]
        if editeur in editor_stats:
            editor_stats[editeur].append(note)
        else:
            editor_stats[editeur] = [note]

In [45]:
del author_stats[np.nan]
del genre_stats[np.nan]
del editor_stats[np.nan]

In [46]:
author_data = {}
for key in author_stats:
    author_data[key] = (np.mean(author_stats[key]),len(author_stats[key]))
genre_data = {}
for key in genre_stats:
    genre_data[key] = (np.mean(genre_stats[key]),len(genre_stats[key]))
editor_data = {}
for key in editor_stats:
    editor_data[key] = (np.mean(editor_stats[key]),len(editor_stats[key]))

In [47]:
#save author_data, genre_data, editor_data as json files
with open(os.path.join(Generation_folder, 'author_data.json'), 'w') as fp:
    json.dump(author_data, fp)
with open(os.path.join(Generation_folder, 'genre_data.json'), 'w') as fp:
    json.dump(genre_data, fp)
with open(os.path.join(Generation_folder, 'editor_data.json'), 'w') as fp:
    json.dump(editor_data, fp)

In [48]:
#on recharge tous les fichiers json pour verifier
with open(os.path.join(Generation_folder, 'titre_info.json'), 'r') as fp:
    titre_info = json.load(fp)
with open(os.path.join(Generation_folder, 'author_data.json'), 'r') as fp:
    author_data = json.load(fp)
with open(os.path.join(Generation_folder, 'genre_data.json'), 'r') as fp:
    genre_data = json.load(fp)
with open(os.path.join(Generation_folder, 'editor_data.json'), 'r') as fp:
    editor_data = json.load(fp)



In [49]:
#A partir du titre de chaque rating, on va extraire le genre, l'auteur et l'editeur et on rajoutes ces informations dans le dataframe ratings
ratings_author = []
nb_rat_auhtor = []
ratings_genre = []
nb_rat_genre = []
ratings_editor = []
nb_rat_editor = []

for i in range(len(ratings_books)):
    titre = ratings_books[i][0]
    if titre in titre_info:
        if titre_info[titre]['Auteurs'] in author_data:
            ratings_author.append(author_data[titre_info[titre]['Auteurs']][0])
            nb_rat_auhtor.append(author_data[titre_info[titre]['Auteurs']][1])
        else:
            ratings_author.append(2.5)
            nb_rat_auhtor.append(0)

        if titre_info[titre]['Genre'] in genre_data:
            ratings_genre.append(genre_data[titre_info[titre]['Genre']][0])
            nb_rat_genre.append(genre_data[titre_info[titre]['Genre']][1])
        else:
            ratings_genre.append(2.5)
            nb_rat_genre.append(0)
            
        if titre_info[titre]['Editeur'] in editor_data:
            ratings_editor.append(editor_data[titre_info[titre]['Editeur']][0])
            nb_rat_editor.append(editor_data[titre_info[titre]['Editeur']][1])
        else:
            ratings_editor.append(2.5)
            nb_rat_editor.append(0)
    else:
        ratings_author.append(0)
        nb_rat_auhtor.append(2.5)
        ratings_genre.append(0)
        nb_rat_genre.append(2.5)
        ratings_editor.append(0)
        nb_rat_editor.append(2.5)

In [50]:
#on rajoute les colonnes dans le dataframe ratings
ratings['auteur_note'] = ratings_author
ratings['auteur_nb'] = nb_rat_auhtor
ratings['genre_note'] = ratings_genre
ratings['genre_nb'] = nb_rat_genre
ratings['editor_note'] = ratings_editor
ratings['editor_nb'] = nb_rat_editor


In [51]:
ratings.to_csv(os.path.join(Generation_folder, 'ratings_final.csv'),index=False)

In [52]:
ratings.head(2)

,Id,Titre,User_id,Nom lecteur,revue/utilité,revue/score,revue/heure,revue/résumé,revue/texte,utilite_num,auteur_note,auteur_nb,genre_note,genre_nb,editor_note,editor_nb
0,1882931173,Its Only Art If Its Well Hung!,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...,1.0,4.0,1.0,4.335691,13149.0,2.500000,0.0
1,0826414346,Dr. Seuss: American Icon,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...,1.0,4.6,10.0,4.258888,107788.0,4.212751,2886.0


In [55]:
rating_formated = ratings.drop(columns=['Id'])  
rating_formated = ratings.dropna(subset=['Titre'])
rating_formated = rating_formated.drop(columns=['User_id'])
rating_formated = rating_formated.drop(columns=['Nom lecteur'])
rating_formated = rating_formated.drop(columns=['revue/utilité'])
rating_formated = rating_formated.drop(columns=['revue/heure'])
rating_formated = rating_formated.dropna(subset=['revue/résumé'])
rating_formated = rating_formated.dropna(subset=['revue/texte'])
#on renomme la colonne revue/score en rating
rating_formated.rename(columns={'revue/score':'rating',"revue/résumé":"summary","revue/texte":"comment"}, inplace=True)
rating_formated.head()

,Titre,rating,summary,comment,utilite_num,auteur_note,auteur_nb,genre_note,genre_nb,editor_note,editor_nb
0,Its Only Art If Its Well Hung!,2,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...,1.0,4.0,1.0,4.335691,13149.0,2.500000,0.0
1,Dr. Seuss: American Icon,2,Really Enjoyed It,I don't care much for Dr. Seuss but after read...,1.0,4.6,10.0,4.258888,107788.0,4.212751,2886.0
2,Dr. Seuss: American Icon,2,Essential for every personal and Public Library,"If people become the books they read and if ""t...",0.909091,4.6,10.0,4.258888,107788.0,4.212751,2886.0
3,Dr. Seuss: American Icon,2,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",1.0,4.6,10.0,4.258888,107788.0,4.212751,2886.0
4,Dr. Seuss: American Icon,2,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...,1.0,4.6,10.0,4.258888,107788.0,4.212751,2886.0


In [56]:
#on sauvegarde le fichier final
rating_formated.to_csv(os.path.join(Generation_folder, 'ratings_formatted.csv'),index=False)